In [ ]:
import tensorflow
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow.keras.layers as layer
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tools import create_split_gen, plot_fitting_history,class_report,find_errors
tensorflow.keras.utils.set_random_seed(812)
tensorflow.config.experimental.enable_op_determinism()

In [ ]:
train_augmentation = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=20,
    zoom_range=0.1,
    horizontal_flip=True,
    width_shift_range=0.1,   
    height_shift_range=0.1 )
train_gen = create_split_gen(train_augmentation, path= "/home/ocelot/Desktop/CNN_RPS/Archive/train")

val_gen = create_split_gen(ImageDataGenerator(rescale = 1./255), path = "/home/ocelot/Desktop/CNN_RPS/Archive/validation")


In [ ]:
early_stop = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
model = tensorflow.keras.models.Sequential([
    layer.InputLayer(shape=(100,150,1)),
    layer.Conv2D(filters=(32), kernel_size=(3,3), activation="relu"),
    layer.MaxPooling2D(pool_size=(2,2)),
    layer.Flatten(),
    layer.Dense(64, activation="relu"),
    layer.Dense(3, activation="softmax")
])

In [ ]:
%%time
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
history=model.fit(train_gen, epochs=100, callbacks=[early_stop], validation_data=val_gen)

In [ ]:
plot_fitting_history(history)

In [ ]:
test_gen = create_split_gen(ImageDataGenerator(rescale = 1./255), "/home/ocelot/Desktop/CNN_RPS/Archive/test", shuffle = False)


In [ ]:
testLoss, testAccuracy = model.evaluate(test_gen)

In [ ]:
pred = model.predict(test_gen)
report = class_report(pred, test_gen)

In [ ]:
report["scissors"]

In [ ]:
report["paper"]

In [ ]:
report["rock"]

In [ ]:
find_errors(pred, test_gen.classes, test_gen)

In [ ]:
model.save("primo.keras")